# Import libraries

In [1]:
import os
import wandb
import torch
from tqdm.auto import tqdm

from torch.utils.data import DataLoader
from torch.optim import AdamW

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from transformers import DataCollatorForSeq2Seq
from transformers import DataCollatorForLanguageModeling

from transformers import get_scheduler

from peft import LoraConfig, TaskType
from peft import get_peft_model

In [2]:
# Import utils from ../src/utils
import sys
sys.path.append('..')

In [3]:
from utils.data import get_mnli
from utils.evaluation import evaluate

# Model

In [9]:
"""
The difference between “it” aka “Instruction Tuned”
and the base model is that the “it” variants are better for chat purposes
since they have been fine-tuned to better understand the instructions
and generate better answers while the base variants are those that have not undergone
under any sort of fine-tuning. They can still generate answers but not as good as the “it” one.

"""
# google/gemma-2b | google/gemma-2b-it | microsoft/phi-2
# Qwen/Qwen1.5-0.5B | Qwen/Qwen1.5-0.5B-Chat
checkpoint_path = "checkpoints/celestial-resonance-10" 
base_model = "microsoft/phi-2"

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"  #{"":0},
)
print(f"Model loaded: {checkpoint_path}")

/home/maparla/anaconda3/envs/ml/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded: checkpoints/celestial-resonance-10


# Tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token # Most LLMs don't have a pad token by default
#data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
max_seq_length = 1024
print(f"Tokenizer loaded: {checkpoint_path}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer loaded: checkpoints/celestial-resonance-10


# Dataset: MNLI

In [11]:
dataset = get_mnli(tokenizer, max_seq_length)

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['class_label', 'idx', 'prompt_length', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 391678
    })
    validation: Dataset({
        features: ['class_label', 'idx', 'prompt_length', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1024
    })
    test_matched: Dataset({
        features: ['class_label', 'idx', 'prompt_length', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9815
    })
    test_mismatched: Dataset({
        features: ['class_label', 'idx', 'prompt_length', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9832
    })
})

## Pytorch dataloader format

In [16]:
batch_size = 1  # number of examples in each batch
inference_batch_size = 1  # number of examples in each batch for inference
max_output_tokens = 32

In [17]:
# Move the data to tensors
dataset.set_format("torch")

In [18]:
val_dataloader = DataLoader(
    dataset["validation"].shuffle(seed=42).select(range(1000)),
    batch_size=inference_batch_size,
    collate_fn=data_collator
)

print(f"Validation dataset size: {len(val_dataloader.dataset)}")

Validation dataset size: 1000


In [19]:
val_preds = evaluate(
    model,
    val_dataloader,
    tokenizer,
    max_output_tokens=max_output_tokens
)
val_correct = sum([1 for p in val_preds if p.y_true == p.y_pred])
val_accuracy = val_correct / len(val_preds)
val_accuracy

Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

/home/maparla/anaconda3/envs/ml/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/maparla/anaconda3/envs/ml/lib/python3.12/site-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


0.857

## Test

### Test Matched

In [20]:
test_matched_dataloader = DataLoader(
    dataset["test_matched"],
    batch_size=inference_batch_size,
    collate_fn=data_collator
)

print(f"Test Matched dataset size: {len(test_matched_dataloader.dataset)}")

Test Matched dataset size: 9815


In [32]:
test_matched_preds = evaluate(
    model,
    test_matched_dataloader,
    tokenizer,
    max_output_tokens=max_output_tokens
)

test_matched_correct = sum([1 for p in test_matched_preds if p.y_true == p.y_pred])
test_matched_accuracy = test_matched_correct / len(test_matched_preds)
test_matched_accuracy

0.8561385634233316

### Test MisMatched

In [41]:
test_mismatched_dataloader = DataLoader(
    dataset["test_mismatched"],
    batch_size=inference_batch_size,
    collate_fn=data_collator
)

print(f"Test Mismatched dataset size: {len(test_mismatched_dataloader.dataset)}")

Test Mismatched dataset size: 9832


In [42]:
test_mismatched_preds = evaluate(
    model,
    test_mismatched_dataloader,
    tokenizer,
    max_output_tokens=max_output_tokens
)

Evaluating:   0%|          | 0/9832 [00:00<?, ?it/s]

/home/maparla/anaconda3/envs/ml/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/maparla/anaconda3/envs/ml/lib/python3.12/site-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [43]:
test_mismatched_correct = sum([1 for p in test_mismatched_preds if p.y_true == p.y_pred])
test_mismatched_accuracy = test_mismatched_correct / len(test_mismatched_preds)
test_mismatched_accuracy

0.8567941415785191